# Generate synpuf disclosure risk

Compare synthetic PUFs trained from a 10% sample, both to the training set and a 10% holdout. Synthetic file (1) is from synthimpute random forests; (2) is from the synthpop R package.

## Setup

### Imports

In [1]:
import pandas as pd
import numpy as np
import synthimpute as si

### Load data

In [2]:
synth = pd.read_csv('~/Downloads/puf_synth_10p_sample.csv')
synthpop = pd.read_csv('~/Downloads/synthpop_samp.csv')
train = pd.read_csv('~/Downloads/puf_10p_sample_train.csv')
test = pd.read_csv('~/Downloads/puf_10p_sample_test.csv')

## Preprocessing

Drop calculated features used as seeds, and drop s006.

In [3]:
synth.drop(['E00100', 'E09600', 'S006'], axis=1, inplace=True)
train.drop(['E00100', 'E09600', 'S006'], axis=1, inplace=True)
test.drop(['E00100', 'E09600', 'S006'], axis=1, inplace=True)
synthpop = synthpop[synth.columns]

In [4]:
synthpop = synthpop[synth.columns]

Check pending https://github.com/MaxGhenis/synthimpute/issues/8.

In [5]:
synth.reset_index(drop=True, inplace=True)
synthpop.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [6]:
# TODO: Do this in the file creation.
def add_subtracted_features(df):
    df['E00600'] = df.E00650 + df.e00600_minus_e00650
    df['E01500'] = df.E01700 + df.e01500_minus_e01700
    df.drop(['e00600_minus_e00650', 'e01500_minus_e01700'], axis=1, inplace=True)
    
add_subtracted_features(train)
add_subtracted_features(test)

## RF

Compare nearest standardized Euclidean distance.

In [7]:
BLOCKS = ['MARS', 'DSI', 'XTOT']  # Reduce after optimizing.

In [8]:
%%time
nearest = si.nearest_synth_train_test(synth, train, test, BLOCKS, metric='euclidean')

Running block 1 of 15...
Running block 2 of 15...
Running block 3 of 15...
Running block 4 of 15...
Running block 5 of 15...
Running block 6 of 15...
Running block 7 of 15...
Running block 8 of 15...
Running block 9 of 15...
Running block 10 of 15...
Running block 11 of 15...
Running block 12 of 15...
Running block 13 of 15...
Running block 14 of 15...
Running block 15 of 15...
Running block 1 of 15...
Running block 2 of 15...
Running block 3 of 15...
Running block 4 of 15...
Running block 5 of 15...
Running block 6 of 15...
Running block 7 of 15...
Running block 8 of 15...
Running block 9 of 15...
Running block 10 of 15...
Running block 11 of 15...
Running block 12 of 15...
Running block 13 of 15...
Running block 14 of 15...
Running block 15 of 15...
CPU times: user 1min 46s, sys: 27.7 s, total: 2min 13s
Wall time: 1min 49s


In [9]:
nearest.to_csv('~/Downloads/nearest_synpuf7.csv')

## synthpop

In [10]:
%%time
nearest2 = si.nearest_synth_train_test(synthpop, train, test, BLOCKS, metric='euclidean')

Running block 1 of 14...
Running block 2 of 14...
Running block 3 of 14...
Running block 4 of 14...
Running block 5 of 14...
Running block 6 of 14...
Running block 7 of 14...
Running block 8 of 14...
Running block 9 of 14...
Running block 10 of 14...
Running block 11 of 14...
Running block 12 of 14...
Running block 13 of 14...
Running block 14 of 14...
Running block 1 of 14...
Running block 2 of 14...
Running block 3 of 14...
Running block 4 of 14...
Running block 5 of 14...
Running block 6 of 14...
Running block 7 of 14...
Running block 8 of 14...
Running block 9 of 14...
Running block 10 of 14...
Running block 11 of 14...
Running block 12 of 14...
Running block 13 of 14...
Running block 14 of 14...
CPU times: user 1min 42s, sys: 22.9 s, total: 2min 5s
Wall time: 1min 42s


In [11]:
nearest2.to_csv('~/Downloads/nearest_synthpop_samp.csv')